In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, Subset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from collections import defaultdict
import numpy as np
import copy
from tqdm import tqdm

# 超参数
NUM_CLIENTS = 10
NUM_ROUNDS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 0.01

# LoRA超参数
LORA_R = 8  # 增加秩以提高表达能力
LORA_ALPHA = 16 # 调整缩放因子

# --- 数据集准备 ---
def prepare_non_iid_data():
    train_data = MNIST(root='./data', train=True, download=True, transform=ToTensor())
    test_data = MNIST(root='./data', train=False, download=True, transform=ToTensor())

    data_by_label = defaultdict(list)
    for img, label in zip(train_data.data, train_data.targets):
        data_by_label[label.item()].append((img, label))

    client_datasets = [[] for _ in range(NUM_CLIENTS)]
    client_labels_per_client = 2
    labels_list = list(data_by_label.keys())
    for i in range(NUM_CLIENTS):
        start_label_index = (i * client_labels_per_client) % len(labels_list)
        labels_to_assign = [labels_list[start_label_index], labels_list[(start_label_index + 1) % len(labels_list)]]
        for label in labels_to_assign:
            client_datasets[i].extend(data_by_label[label])

    client_dataloaders = []
    client_data_sizes = []
    for data in client_datasets:
        imgs = torch.stack([d[0] for d in data]).unsqueeze(1).float() / 255.0
        labels = torch.tensor([d[1] for d in data]).long()
        dataset = TensorDataset(imgs, labels)
        client_dataloaders.append(DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True))
        client_data_sizes.append(len(dataset))

    test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

    return client_dataloaders, client_data_sizes, test_dataloader

# --- LoRA 模块和模型定义（手搓） ---
class LoRA_Linear(nn.Module):
    def __init__(self, in_features, out_features, rank, alpha):
        super().__init__()
        self.rank = rank
        self.alpha = alpha

        # 基础线性层
        self.base_layer = nn.Linear(in_features, out_features)

        # LoRA矩阵初始化：A随机，B为零
        self.lora_A = nn.Parameter(torch.randn(in_features, rank) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(rank, out_features))

        self.scaling = self.alpha / self.rank

        # 冻结基础层权重
        self.base_layer.weight.requires_grad = False
        self.base_layer.bias.requires_grad = False

    def forward(self, x):
        # 计算基础层的输出
        base_output = self.base_layer(x)

        # 计算 LoRA 模块的更新
        lora_update = (x @ self.lora_A @ self.lora_B) * self.scaling

        return base_output + lora_update

class LoRAMNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = LoRA_Linear(28*28, 128, LORA_R, LORA_ALPHA)
        self.relu1 = nn.ReLU()
        self.fc2 = LoRA_Linear(128, 64, LORA_R, LORA_ALPHA)
        self.relu2 = nn.ReLU()
        self.fc3 = LoRA_Linear(64, 10, LORA_R, LORA_ALPHA)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# --- 评估和训练函数 ---
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def train_federated_model(global_model, client_dataloaders, client_data_sizes, aggregation_method):
    history = []

    for round_num in range(NUM_ROUNDS):
        client_lora_weights = []

        for i, dataloader in enumerate(client_dataloaders):
            client_model = copy.deepcopy(global_model)
            optimizer = torch.optim.SGD(
                [p for p in client_model.parameters() if p.requires_grad],
                lr=LEARNING_RATE
            )
            criterion = nn.CrossEntropyLoss()

            client_model.train()
            for epoch in range(LOCAL_EPOCHS):
                for imgs, labels in dataloader:
                    optimizer.zero_grad()
                    outputs = client_model(imgs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

            # 提取所有LoRA层的权重
            lora_weights = {}
            for name, module in client_model.named_modules():
                if isinstance(module, LoRA_Linear):
                    lora_weights[name] = {
                        'A': module.lora_A.detach().clone(),
                        'B': module.lora_B.detach().clone()
                    }
            client_lora_weights.append(lora_weights)

        # 聚合步骤
        if aggregation_method == 'zero_padding':
            global_lora_updates = defaultdict(lambda: defaultdict(list))
            for client_weights in client_lora_weights:
                for name, weights in client_weights.items():
                    global_lora_updates[name]['A'].append(weights['A'])
                    global_lora_updates[name]['B'].append(weights['B'])

            # 聚合 LoRA 矩阵 A 和 B
            aggregated_weights = {}
            for name, weights_list in global_lora_updates.items():
                # 聚合 A
                max_rank_A = max(w.shape[1] for w in weights_list['A'])
                padded_A_list = [torch.zeros(w.shape[0], max_rank_A) if w.shape[1] < max_rank_A else w for w in weights_list['A']]
                for i, w in enumerate(weights_list['A']):
                    if w.shape[1] < max_rank_A:
                        padded_A_list[i][:, :w.shape[1]] = w

                # 聚合 B
                max_rank_B = max(w.shape[0] for w in weights_list['B'])
                padded_B_list = [torch.zeros(max_rank_B, w.shape[1]) if w.shape[0] < max_rank_B else w for w in weights_list['B']]
                for i, w in enumerate(weights_list['B']):
                    if w.shape[0] < max_rank_B:
                        padded_B_list[i][:w.shape[0], :] = w

                avg_A = torch.stack(padded_A_list).mean(dim=0)
                avg_B = torch.stack(padded_B_list).mean(dim=0)
                aggregated_weights[name] = {'A': avg_A, 'B': avg_B}

        elif aggregation_method == 'weighted_averaging':
            total_data_size = sum(client_data_sizes)
            aggregated_weights = {}

            # 初始化聚合后的权重
            for name in client_lora_weights[0].keys():
                avg_A = torch.zeros_like(client_lora_weights[0][name]['A'])
                avg_B = torch.zeros_like(client_lora_weights[0][name]['B'])
                for i, client_weights in enumerate(client_lora_weights):
                    weight = client_data_sizes[i] / total_data_size
                    avg_A += client_weights[name]['A'] * weight
                    avg_B += client_weights[name]['B'] * weight
                aggregated_weights[name] = {'A': avg_A, 'B': avg_B}

        # 更新全局模型的LoRA权重
        for name, module in global_model.named_modules():
            if isinstance(module, LoRA_Linear):
                module.lora_A.data = aggregated_weights[name]['A']
                module.lora_B.data = aggregated_weights[name]['B']

        # 评估全局模型
        accuracy = evaluate_model(global_model, test_dataloader)
        print(f"聚合方法: {aggregation_method} | 回合 {round_num + 1}/{NUM_ROUNDS} 准确率: {accuracy:.2f}%")
        history.append(accuracy)
    return history

if __name__ == '__main__':
    client_dataloaders, client_data_sizes, test_dataloader = prepare_non_iid_data()

    # 零值填充聚合
    print("--- 开始使用零值填充聚合 ---")
    global_model_zp = LoRAMNISTModel()
    history_zp = train_federated_model(global_model_zp, client_dataloaders, client_data_sizes, 'zero_padding')

    print("\n" + "="*50 + "\n")

    # 加权平均聚合
    print("--- 开始使用加权平均聚合 ---")
    global_model_wa = LoRAMNISTModel()
    history_wa = train_federated_model(global_model_wa, client_dataloaders, client_data_sizes, 'weighted_averaging')

    print("\n--- 最终结果对比 ---")
    print("零值填充聚合的准确率历史:", [f"{acc:.2f}%" for acc in history_zp])
    print("加权平均聚合的准确率历史:", [f"{acc:.2f}%" for acc in history_wa])

--- 开始使用零值填充聚合 ---
聚合方法: zero_padding | 回合 1/50 准确率: 8.97%
聚合方法: zero_padding | 回合 2/50 准确率: 11.44%
聚合方法: zero_padding | 回合 3/50 准确率: 16.98%
聚合方法: zero_padding | 回合 4/50 准确率: 42.10%
聚合方法: zero_padding | 回合 5/50 准确率: 53.66%
聚合方法: zero_padding | 回合 6/50 准确率: 55.66%
聚合方法: zero_padding | 回合 7/50 准确率: 56.64%
聚合方法: zero_padding | 回合 8/50 准确率: 57.55%
聚合方法: zero_padding | 回合 9/50 准确率: 57.69%
聚合方法: zero_padding | 回合 10/50 准确率: 57.74%
聚合方法: zero_padding | 回合 11/50 准确率: 57.40%
聚合方法: zero_padding | 回合 12/50 准确率: 58.05%
聚合方法: zero_padding | 回合 13/50 准确率: 57.24%
聚合方法: zero_padding | 回合 14/50 准确率: 58.98%
聚合方法: zero_padding | 回合 15/50 准确率: 58.04%
聚合方法: zero_padding | 回合 16/50 准确率: 58.23%
聚合方法: zero_padding | 回合 17/50 准确率: 59.62%
聚合方法: zero_padding | 回合 18/50 准确率: 58.78%
聚合方法: zero_padding | 回合 19/50 准确率: 59.45%
聚合方法: zero_padding | 回合 20/50 准确率: 59.10%
聚合方法: zero_padding | 回合 21/50 准确率: 58.32%
聚合方法: zero_padding | 回合 22/50 准确率: 60.29%
聚合方法: zero_padding | 回合 23/50 准确率: 59.49%
聚合方法: zero_padding | 回合 2

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from collections import defaultdict
import numpy as np
import copy
from tqdm import tqdm

# Hyperparameters
NUM_CLIENTS = 10
NUM_ROUNDS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 0.01

# LoRA hyperparameters
LORA_R = 8
LORA_ALPHA = 16

# --- Data Preparation ---
def prepare_non_iid_data():
    train_data = MNIST(root='./data', train=True, download=True, transform=ToTensor())
    test_data = MNIST(root='./data', train=False, download=True, transform=ToTensor())

    data_by_label = defaultdict(list)
    for img, label in zip(train_data.data, train_data.targets):
        data_by_label[label.item()].append((img, label))

    client_datasets = [[] for _ in range(NUM_CLIENTS)]
    client_labels_per_client = 2
    labels_list = list(data_by_label.keys())
    for i in range(NUM_CLIENTS):
        start_label_index = (i * client_labels_per_client) % len(labels_list)
        labels_to_assign = [labels_list[start_label_index], labels_list[(start_label_index + 1) % len(labels_list)]]
        for label in labels_to_assign:
            client_datasets[i].extend(data_by_label[label])

    client_dataloaders = []
    client_data_sizes = []
    for data in client_datasets:
        imgs = torch.stack([d[0] for d in data]).unsqueeze(1).float() / 255.0
        labels = torch.tensor([d[1] for d in data]).long()
        dataset = TensorDataset(imgs, labels)
        client_dataloaders.append(DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True))
        client_data_sizes.append(len(dataset))

    test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

    return client_dataloaders, client_data_sizes, test_dataloader

# --- LoRA Module and Model Definition ---
class LoRA_Linear(nn.Module):
    def __init__(self, in_features, out_features, rank, alpha):
        super().__init__()
        self.rank = rank
        self.alpha = alpha

        self.base_layer = nn.Linear(in_features, out_features)

        self.lora_A = nn.Parameter(torch.randn(in_features, rank) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(rank, out_features))

        self.scaling = self.alpha / self.rank
        self.base_layer.weight.requires_grad = False
        self.base_layer.bias.requires_grad = False

    def forward(self, x):
        base_output = self.base_layer(x)
        lora_update = (x @ self.lora_A @ self.lora_B) * self.scaling
        return base_output + lora_update

class LoRAMNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = LoRA_Linear(28*28, 128, LORA_R, LORA_ALPHA)
        self.relu1 = nn.ReLU()
        self.fc2 = LoRA_Linear(128, 64, LORA_R, LORA_ALPHA)
        self.relu2 = nn.ReLU()
        self.fc3 = LoRA_Linear(64, 10, LORA_R, LORA_ALPHA)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# --- Evaluation and Training Functions ---
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            outputs = model(imgs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def train_federated_model(global_model, client_dataloaders, client_data_sizes, aggregation_method, test_dataloader):
    history = []

    for round_num in range(NUM_ROUNDS):
        print(f"--- Federated Round: {round_num + 1}/{NUM_ROUNDS} ---")
        client_lora_weights = []
        local_accuracies = []

        for i, dataloader in enumerate(tqdm(client_dataloaders, desc="Client Local Training")):
            client_model = copy.deepcopy(global_model)
            optimizer = torch.optim.SGD(
                [p for p in client_model.parameters() if p.requires_grad],
                lr=LEARNING_RATE
            )
            criterion = nn.CrossEntropyLoss()

            client_model.train()
            for epoch in range(LOCAL_EPOCHS):
                for imgs, labels in dataloader:
                    optimizer.zero_grad()
                    outputs = client_model(imgs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

            local_acc = evaluate_model(client_model, dataloader)
            local_accuracies.append(local_acc)

            lora_weights = {}
            for name, module in client_model.named_modules():
                if isinstance(module, LoRA_Linear):
                    lora_weights[name] = {
                        'A': module.lora_A.detach().clone(),
                        'B': module.lora_B.detach().clone()
                    }
            client_lora_weights.append(lora_weights)

        print(f"Average Local Accuracy (Round {round_num + 1}): {np.mean(local_accuracies):.2f}%")

        # Aggregation step
        if aggregation_method == 'zero_padding':
            global_lora_updates = defaultdict(lambda: defaultdict(list))
            for client_weights in client_lora_weights:
                for name, weights in client_weights.items():
                    global_lora_updates[name]['A'].append(weights['A'])
                    global_lora_updates[name]['B'].append(weights['B'])

            aggregated_weights = {}
            for name, weights_list in global_lora_updates.items():
                # Aggregate A
                max_rank_A = max(w.shape[1] for w in weights_list['A'])
                padded_A_list = [torch.zeros(w.shape[0], max_rank_A) if w.shape[1] < max_rank_A else w for w in weights_list['A']]
                for i, w in enumerate(weights_list['A']):
                    if w.shape[1] < max_rank_A:
                        padded_A_list[i][:, :w.shape[1]] = w

                # Aggregate B
                max_rank_B = max(w.shape[0] for w in weights_list['B'])
                padded_B_list = [torch.zeros(max_rank_B, w.shape[1]) if w.shape[0] < max_rank_B else w for w in weights_list['B']]
                for i, w in enumerate(weights_list['B']):
                    if w.shape[0] < max_rank_B:
                        padded_B_list[i][:w.shape[0], :] = w

                avg_A = torch.stack(padded_A_list).mean(dim=0)
                avg_B = torch.stack(padded_B_list).mean(dim=0)
                aggregated_weights[name] = {'A': avg_A, 'B': avg_B}

        elif aggregation_method == 'weighted_averaging':
            total_data_size = sum(client_data_sizes)
            aggregated_weights = {}

            for name in client_lora_weights[0].keys():
                avg_A = torch.zeros_like(client_lora_weights[0][name]['A'])
                avg_B = torch.zeros_like(client_lora_weights[0][name]['B'])
                for i, client_weights in enumerate(client_lora_weights):
                    weight = client_data_sizes[i] / total_data_size
                    avg_A += client_weights[name]['A'] * weight
                    avg_B += client_weights[name]['B'] * weight
                aggregated_weights[name] = {'A': avg_A, 'B': avg_B}

        # Update global model LoRA weights
        for name, module in global_model.named_modules():
            if isinstance(module, LoRA_Linear):
                module.lora_A.data = aggregated_weights[name]['A']
                module.lora_B.data = aggregated_weights[name]['B']

        # Evaluate global model
        accuracy = evaluate_model(global_model, test_dataloader)
        print(f"Global Accuracy (Round {round_num + 1}): {accuracy:.2f}%")
        history.append(accuracy)
    return history

if __name__ == '__main__':
    client_dataloaders, client_data_sizes, test_dataloader = prepare_non_iid_data()

    print("--- Starting with Zero-Padding Aggregation ---")
    global_model_zp = LoRAMNISTModel()
    history_zp = train_federated_model(global_model_zp, client_dataloaders, client_data_sizes, 'zero_padding', test_dataloader)

    print("\n" + "="*50 + "\n")

    print("--- Starting with Weighted Averaging Aggregation ---")
    global_model_wa = LoRAMNISTModel()
    history_wa = train_federated_model(global_model_wa, client_dataloaders, client_data_sizes, 'weighted_averaging', test_dataloader)

    print("\n--- Final Results Comparison ---")
    print("Accuracy History (Zero-Padding):", [f"{acc:.2f}%" for acc in history_zp])
    print("Accuracy History (Weighted Averaging):", [f"{acc:.2f}%" for acc in history_wa])

--- Starting with Zero-Padding Aggregation ---
--- Federated Round: 1/50 ---


Client Local Training: 100%|██████████| 10/10 [00:37<00:00,  3.75s/it]


Average Local Accuracy (Round 1): 98.73%
Global Accuracy (Round 1): 14.15%
--- Federated Round: 2/50 ---


Client Local Training: 100%|██████████| 10/10 [00:37<00:00,  3.80s/it]


Average Local Accuracy (Round 2): 99.10%
Global Accuracy (Round 2): 14.07%
--- Federated Round: 3/50 ---


...

Average Local Accuracy (Round 48): 99.60%
Global Accuracy (Round 48): 41.58%
--- Federated Round: 49/50 ---


Client Local Training: 100%|██████████| 10/10 [00:39<00:00,  3.95s/it]


Average Local Accuracy (Round 49): 99.85%
Global Accuracy (Round 49): 38.96%
--- Federated Round: 50/50 ---


Client Local Training: 100%|██████████| 10/10 [00:39<00:00,  3.94s/it]


Average Local Accuracy (Round 50): 99.84%
Global Accuracy (Round 50): 40.76%

--- Final Results Comparison ---
Accuracy History (Zero-Padding): ['14.15%', '14.07%', '29.49%', '34.76%', '37.50%', '39.80%', '39.42%', '42.28%', '42.01%', '44.60%', '42.02%', '43.28%', '45.27%', '41.59%', '42.83%', '42.11%', '42.90%', '42.65%', '41.19%', '43.95%', '42.01%', '43.63%', '44.67%', '44.46%', '39.76%', '45.82%', '47.53%', '47.60%', '45.96%', '47.12%', '47.26%', '47.72%', '48.83%', '46.55%', '49.56%', '50.18%', '48.53%', '48.77%', '49.82%', '48.76%', '50.00%', '49.94%', '49.83%', '48.36%', '50.46%', '48.35%', '50.03%', '46.47%', '50.23%', '46.12%']
Accuracy History (Weighted Averaging): ['20.67%', '24.92%', '23.81%', '24.59%', '25.86%', '25.08%', '27.39%', '25.41%', '27.28%', '25.13%', '27.91%', '25.12%', '24.38%', '27.74%', '28.05%', '30.60%', '31.27%', '33.42%', '31.53%', '30.32%', '31.57%', '31.62%', '31.16%', '32.58%', '30.87%', '30.63%', '31.35%', '30.79%', '32.99%', '31.78%', '30.65%', '29.6